In [4]:
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter
from collections import defaultdict
from pprint import pprint
from nltk import sent_tokenize
punctuation += "«»—…“”"
punct = set(punctuation)
import copy


from sklearn.metrics import classification_report, accuracy_score

In [5]:
bad = open('sents_with_mistakes.txt.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt.txt', encoding='utf8').read().splitlines()

In [6]:
print(bad[2])
print(true[2])

Пояним эту мысль.
Поясним эту мысль


In [7]:
def prepare_sents(sents):
    tokenized = []
    for sent in sents:
        sent = sent.lower()
        tokens = sent.split()
        tokens = [re.sub('(^\W+|\W+$)', '', token) for token in tokens if (set(token)-punct)]
        tokenized.append(list(tokens))
    return tokenized

In [8]:
bad_tokens = prepare_sents(bad)
true_tokens = prepare_sents(true)

In [6]:
print(bad_tokens[1])

['опофеозом', 'дня', 'для', 'меня', 'сегодня', 'стала', 'фраза', 'услышанная', 'в', 'новостях']


In [9]:
import gzip
import csv

In [10]:
corpus = open('corpus_5000.txt', 'w', encoding = 'utf-8')
with gzip.open('lenta-ru-news.csv.gz', 'rt', encoding='utf-8') as archive:
    reader = csv.reader(archive, delimiter=',', quotechar='"')
    for i, line in enumerate(reader):
        if i < 5000: 
            corpus.write(line[2].replace('\xa0', ' ') + '\n')

In [11]:
def normalize(text):
    normalized_text = [(word.strip(punctuation)) for word in text.lower().split()]
    normalized_text = [word for word in normalized_text if word]
    return normalized_text

In [12]:
import string
import nltk
from nltk import sent_tokenize
nltk.download('punkt')
punctuation = string.punctuation+ "«»—…“”"
punct = set(punctuation)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lizan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [13]:
corpus = []
for text in open('corpus_5000.txt', encoding = 'utf-8').read().splitlines():
    sents = sent_tokenize(text)
    norm_sents = [['<START>'] + ['<START>'] + normalize(sent) + ['<END>'] + ['<END>'] for sent in sents]
    corpus += norm_sents

In [ ]:
## Нам всё равно понадобится список токенов без повторений, почему бы не посчитать частотность слов и заодно Ngram уже сейчас

In [14]:
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [15]:
unigrams= Counter()
bigrams = Counter()
trigrams = Counter()

for sentence in corpus:
    unigrams.update(sentence)
    bigrams.update(ngrammer(sentence))
    trigrams.update(ngrammer(sentence, 3))

In [16]:
def del_letter(word, n=0):
    word_L = word[:n]
    word_R = word[n:]
    word_afterdel = word_L + word_R[1:]
    return word_afterdel

In [91]:
def clear_multiplied(word): #один раз превращает три повторяющиеся буквы в две
    delword = []
    clear_word = word
    for i in range(len(word)-3):
        if word[i] == word[i+1] and word[i] == word[i+2]:
            clear_word = del_letter(clear_word, i) 
            break
    return clear_word

In [86]:
print(clear_multiplied('хорошоооооо')) #убирает часть из повторяющихся букв

хорошооооо


In [104]:
def clear_all_multiplied(word):
    word2 = word
    while clear_multiplied(clear_multiplied(word2)) != clear_multiplied(word2):
        word2 = clear_multiplied(clear_multiplied(word2))
    return word2

In [106]:
#прошу прощения, я так и не поняла, почему функция убрает не все случаи, когда подряд идёт юольше двух одинаковых символов,
#но часть она убирает, так что применю. В примере выше все буквы о точно кириллические.
bad_tokens_multiclean = bad_tokens
for i in bad_tokens_multiclean:
    for token in i:
        new_token = clear_all_multiplied(token)
        if new_token != token:
            print(token, new_token)
            token = new_token

оччччень оччень
каааак каак
оооочень оочень
оооочень оочень
страааашно страашно
оооочень оочень
оооочень оочень
доолго-дооолго-доооолго доолго-доолго-дооолго
оооочень оочень
мааааммоооочкииии мааммоочкииии
ээээххх ээххх
ооооочень ооочень
ооооочень ооочень
ооооочень ооочень
оооочень оочень
ооооочень ооочень
прррроклятые прроклятые
каааак каак
оооочень оочень
ооооочень ооочень
оооочень оочень
спаааасиб спаасиб
оооочень оочень
кошшшшка кошшка
каааак каак
воооот воот
покажиииии покажиии


In [107]:
def corpus_del(corpus):
    corpusdict = defaultdict(set)
    for word in corpus:
        for n in range(len(word)):
            word1del = del_letter(word, n)
            corpusdict[word1del].add(word)
    return(corpusdict)

In [108]:
corpusdict = corpus_del(unigrams.elements())

In [109]:
print(corpusdict['коа'])

{'кожа', 'копа', 'коба', 'кота', 'коза', 'коап', 'кора', 'кома', 'кода', 'коса'}


In [110]:
unigrams_set = set(unigrams.elements())

In [111]:
def symspell2dels(text, corpus, corpusdict):
    textset = set(text)
    corrections = {}
    for word in textset:
        correct = set()
        if word not in corpus:
            delword = set()
            for n in range(len(word)):
                word1del = del_letter(word, n)
                delword.add(word1del) 
            for seq in delword:
                if seq in corpus:
                    correct.add(seq)
                if seq in corpusdict:
                    for correctword in corpusdict[seq]:
                        correct.add(correctword)
        if correct:
            corrections[word] = correct
    return corrections  

In [114]:
bad_list = []
for i in bad_tokens_multiclean:
    for j in i:
        bad_list.append(j)

In [115]:
corrections = symspell2dels(bad_list, unigrams_set, corpusdict)

In [116]:
n = len(bad_tokens_multiclean)
corrected_randomoption = [[] for i in range(n)]
for i in range(n):
    for token in bad_tokens_multiclean[i]:
        if token not in corrections:
                corrected_randomoption[i].append(token)
        else:
            existing_word = random.choice(list(corrections[token]))
            corrected_randomoption[i].append(existing_word)

In [117]:
print(corrected_randomoption[1])

['апофеозом', 'дня', 'для', 'меня', 'сегодня', 'стала', 'фраза', 'услышанная', 'в', 'новостях']


# Оценим работу спелчекера, выбирающего одно из наиболее близких слов

In [120]:
def metrics(true, bad, corrected):
    total_mistaken = 0
    really_corrected = 0
    not_corrected = 0
    broken = 0
    correct_at_bad = 0
    wrong_at_bad = 0
    for i in range(len(true)):
        if len(true[i]) == len(bad[i]): #я не исправляла ошибки в слитно-раздельном написании
            for j in range(len(true[i])):
                true_word = true[i][j]
                bad_word = bad[i][j]
                corrected_word = corrected[i][j]
                if true_word != bad_word:
                    wrong_at_bad += 1
                    if true_word == corrected_word:
                        really_corrected += 1
                    else:
                        total_mistaken += 1
                        not_corrected += 1
                else:
                    correct_at_bad += 1
                    if true_word != corrected_word:
                        total_mistaken += 1
                        broken += 1
    metrics_dict = {}
    metrics_dict['%mistakes'] = total_mistaken/(correct_at_bad + wrong_at_bad)
    metrics_dict['%corrected'] = really_corrected/wrong_at_bad * 100
    metrics_dict['%broken'] = broken/correct_at_bad*100
    return(metrics_dict)       

In [121]:
dist_metrics_random = metrics(true_tokens, bad_tokens, corrected_randomoption)

In [122]:
print(dist_metrics_random) #при перезапуске кода проценты будут немного меняться

{'%mistakes': 0.1567783891945973, '%corrected': 27.251552795031053, '%broken': 7.235557597335478}


# Будем выбирать слово в соответствии с частотностью триграм

In [123]:
#N-грамы посчитали выше
def p_max_words(ngrams, prevngrams, prevngram, options, p_max0 = 0, p_max_words0 = []):
    p_max_words = p_max_words0
    p_max = p_max0
    for option in options:
        try:
            p = ngrams[prevngram + ' ' + option]/prevngrams[prevngram]
        except ZeroDivisionError:
                p = 0
        if p > p_max:
            p_max = p
            p_max_words = []
        if p == p_max:
            p_max_words.append(option)
    return(p_max_words)

In [125]:
def trigram_correct_sentence(sentence, corrections, bigrams, trigrams):
    corrected_sent = ['<start>', '<start>']
    for word in sentence:
        if word not in corrections:
            corrected_sent.append(word)
        else:
            correction = list(corrections[word])
            if len(correction) == 1:
                corrected_sent.append(correction[0])
            else:
                try:
                    corrected_bigram = corrected_sent[-2] + ' ' + corrected_sent[-1]
                except:
                    print(isinstance(corrected_sent[-2] , str))
                max_p = 0
                max_p_words = []
                p_max_trigram = p_max_words(trigrams, bigrams, corrected_bigram, correction)
                try:
                    chosen = random.choice(list(p_max_trigram))  #берём любое слово из самых вероятных
                    corrected_sent.append(chosen)
                except:
                    corrected_sent.append(word)  #если по какой-то причине список кандидатов оказался пустым,
    return corrected_sent                           #добавится изначальное слово

In [126]:
corrected_text_trigrams = [trigram_correct_sentence(sent, corrections, bigrams, trigrams)[2:] for sent in bad_tokens]
#не берём тэги начала предложения

In [127]:
dist_metrics_trigram = metrics(true_tokens, bad_tokens, corrected_text_trigrams)
print(dist_metrics_trigram) 
#процент broken не изменился, так как для этой метрики неважно, на что заменено правильное слово
#процент corrected понизился

{'%mistakes': 0.16258129064532267, '%corrected': 22.748447204968944, '%broken': 7.235557597335478}
